In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm import tqdm
import os

In [4]:
qdf= pd.read_csv('../data/from-smiles/Lipophilicity.csv')
smiles = qdf['smiles']

three_ds = []
failed_three_ds = []
three_ds_path = '../data/from-smiles/Lipophilicity-3d/'
opt_three_ds = []
failed_opt_three_ds = []
opt_three_ds_path = '../data/from-smiles/Lipophilicity-3d-opt/'
two_ds = []
two_ds_path = '../data/from-smiles/Lipophilicity-2d/'
sdf_file = 'Lipophilicity.sdf'
csv_file = 'Lipophilicity.sdf.csv'

target_columns = ['exp']

for smile in tqdm(smiles):
    try:
        mol = Chem.MolFromSmiles(smile)
        mol = Chem.AddHs(mol)
        mol.SetProp('_Name', smile)
        
        AllChem.Compute2DCoords(mol)

        # Save a copy of the 2D molecule
        two_ds.append(mol.__copy__())

        if (AllChem.EmbedMolecule(mol, randomSeed = 0xf00d, maxAttempts = 100) == -1):
            failed_three_ds.append(smile)
            continue
        # Save 3D molecule
        three_ds.append(mol.__copy__())

        if AllChem.MMFFOptimizeMolecule(mol) == -1:
            failed_opt_three_ds.append(smile)
            continue

        # Save optimized 3D molecule
        opt_three_ds.append(mol.__copy__())
    except Exception as e:
        print(e)
        continue

if not os.path.exists(three_ds_path):
    os.makedirs(three_ds_path)

if not os.path.exists(opt_three_ds_path):
    os.makedirs(opt_three_ds_path)

if not os.path.exists(two_ds_path):
    os.makedirs(two_ds_path)

with Chem.SDWriter(two_ds_path + sdf_file) as writer:
    for mol in two_ds:
        writer.write(mol)
qdf.drop(columns=qdf.columns.difference(target_columns), inplace=False).to_csv(two_ds_path + csv_file, index=False)

with Chem.SDWriter(three_ds_path + sdf_file) as writer:
    for mol in three_ds:
        writer.write(mol)
qdf.drop(columns=qdf.columns.difference(target_columns), inplace=False).to_csv(three_ds_path + csv_file, index=False)


with Chem.SDWriter(opt_three_ds_path + sdf_file) as writer:
    for mol in opt_three_ds:
        writer.write(mol)
qdf.drop(columns=qdf.columns.difference(target_columns), inplace=False).to_csv(opt_three_ds_path + csv_file, index=False)


 37%|███▋      | 1561/4200 [01:37<02:08, 20.47it/s][14:59:33] UFFTYPER: Unrecognized atom type: Se2+2 (3)
[14:59:33] UFFTYPER: Unrecognized atom type: Se2+2 (3)
100%|██████████| 4200/4200 [04:21<00:00, 16.05it/s]


In [5]:
sdf = Chem.SDMolSupplier('data/from-smiles/gdb8-3d/gdb8.sdf')

In [22]:
try:
    mol = Chem.MolFromSmiles('[H]C([H])([H])C12C([H])([H])N(C1([H])[H])C2([H])[H]')
    mol = Chem.AddHs(mol)
    print(AllChem.EmbedMolecule(mol))
    print(AllChem.EmbedMolecule(mol))
except:
    print('Failed to generate conformer for unrecognized molecules')

-1
-1
